# Goal
- winston benchmark を解く

## 一時目標 
- toy problem を手動の導出なしで解く
- 


### TO DO
- 演算の定義
- 記号と処理の対応付け
- 演算の順番の実装
- predicate に対しても p_j を割り当て （ex. mammal(x) =: p_2(x）)
- 上から読み込んでいったとしたら 初めてでてきた predicate を認識して


### Notes

- 解釈の順番 否定を先
- 論理式（KB）の書き方を少し制限する
-
- とりあえず，分配法則を使用した書き方を禁止する
- 2重否定もとりあえず許さない（許してもよさそう）
-
- p(x) ○ q(x) 以上の formula を解析しようとするともう少し抽象度をあげなければいけない
- 受け取った formula を空白で区切って要素ごとに分解しリストを作成
- 演算子を認識して前から順に演算
- 演算を終えるごとに，リストを更新し，リストの要素数が 1 かつ，それが演算子でなくなるまで演算を繰り返す
- 
- 
- 与えられた formula の構文チェッカーを作成しても良いかもしれない
- 
- [p_1(x_test), '→', '¬', p_2(x_test), '→', '¬', p_3(x_test)] のような str と cp.Variable が混ざったリストに対して .count や .index ができない
- ループなどを使ってカウントするしか無い
- 
- p を cvxpy で書いたら affine ではなくなった
- おそらく 含意 がわるい
- min, max を取る操作を含むと affine ではなくなる
- min(x, y) >= val  <=>  x >= val と y >= val
- max(x, y) <= val  <=>  x >= val と y <= val
- 
- どこまで式変形すれば affine のまま不等式が定義できるか
- 結果として制約の数が増えても良い
- min(x, y) >= val  <=>  x >= val と y >= val を使って min を消す
- ~~max(x, y) <= val  <=>  x <= val と y <= val を使って max を消す~~
- logical <= ξ なので，1 - min(・, ・) として，不等式を並べる方向式変形する
- x '∨' y <= ξ が出てきたらどうするか（Example 3 がその例）
- 
- 矛盾する制約（ξ <= -1 and ξ <= 1）のようながある時どうなるのか
- とりあえず上は考えない （convex (concave) fragments の中で閉じてるはずなので）
- 
- 
- 制約が affine でなくなるのを防止するために不等式を分割すると上手く計算できることが確認できた
- あとは formula の変換プログラムを完成させるだけ
- 
- 排他的論理和(ではない) ⊕ の分配法則を勘違いしていた 
- x ⊕ (y ∧ z) = (x ⊕ y) ∧ (x ⊕ z)
- 
- 分配法則を処理できるようにしないといけないかもしれない 
- 
- ホーン節とか，誤植とか
- 定義をしっかり覚えておかしいところは気づかないといけない
- ホーン節チェッカーを作成しても良いかもしれない
- 
- 否定を先に計算しなくても良いようにできると良いんだけど
- 
- 演算子の変換の際には string のまま変換したほうが良さそう
- 足し算はリスト同士の足し算
-
- negation だけは適用する要素にひっつけて書くとか
- 分配法則の実装と合わせて
- 
- もしかしたら，後ろから分配法則で処理していくのが一貫してて良いのではないか
-  
- ホーン節 チェッカー
- とりあえず min と max が混在するようだとおかしい，
- 特に，logical constraints の特徴上，KB 中の formula を変形して最後に max が残らないようにする
- 
- もしあれば，'→' implication を境にして，それより前に min を生成するような演算があるとそれは formula として正しくない（なぜなら，'→' の前全体に '¬' 否定がかかって，min が裏返って max になってしまうため）
- '→' implication の後に max があるのもいけない，なぜなら裏返って min になることなく，最後まで max が残ってしまうため
- 
- 演算子に追加で，「+」「-」も扱えたほうが良さそう
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 



#### 目印になりそうなもの

- 半角スペース
- ()
- () の左に隣接する文字列
- 

In [2]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

from utils.misc import *
from utils.misc import Predicate, count_neg, get_first_neg_index, convert_formula
from utils.misc import count_
from utils.operators import negation
from utils.operators import weak_conjunction, strong_conjunction
from utils.operators import weak_disjunction, strong_disjunction
from utils.operators import implication

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# load and convert data, describe problem settings, etc
data_dir_path = './inputs/toy_data/'
path_L1 = os.path.join(data_dir_path, 'L1.csv')
path_L2 = os.path.join(data_dir_path, 'L2.csv')
path_L3 = os.path.join(data_dir_path, 'L3.csv')
path_U = os.path.join(data_dir_path, 'U.csv')

df_L1 = pd.read_csv(path_L1, index_col=0)
df_L2 = pd.read_csv(path_L2, index_col=0)
df_L3 = pd.read_csv(path_L3, index_col=0)
df_U = pd.read_csv(path_U, index_col=0)

L1 = np.array(df_L1)
L2 = np.array(df_L2)
L3 = np.array(df_L3)

L = np.stack([L1, L2, L3]) # data for pointwise constraint
U = np.array(df_U) # data for logical constriant

In [8]:
symbols_1 = ['¬', '∧', '∨', '⊗', '⊕', '→']

operations = [
    negation,
    weak_conjunction,
    weak_disjunction,
    strong_conjunction,
    strong_disjunction,
    implication
]

symbols_1_semanticized = {s: o for s, o in zip(symbols_1, operations)}

In [9]:
# path_rules = os.path.join(data_dir_path, 'rules.txt')
path_rules = os.path.join(data_dir_path, 'rules_2.txt')
knowledge_base = []

with open(path_rules, 'r') as f:
    for line in f:
        knowledge_base.append(line.split())

display(knowledge_base)

len_h = len(knowledge_base)

[['hair(x)', '→', 'mammal(x)'],
 ['milk(x)', '→', 'mammal(x)'],
 ['feather(x)', '→', 'bird(x)'],
 ['layeggs(x)', '→', 'bird(x)'],
 ['mammal(x)', '⊗', 'meat(x)', '→', 'carnivore(x)'],
 ['mammal(x)',
  '⊗',
  'pointedteeth(x)',
  '⊗',
  'claws(x)',
  '⊗',
  'forwardeyes(x)',
  '→',
  'carnivore(x)'],
 ['mammal(x)', '⊗', 'hoofs(x)', '→', 'ungulate(x)'],
 ['mammal(x)', '⊗', 'cud(x)', '→', 'ungulate(x)'],
 ['carnivore(x)', '⊗', 'tawny(x)', '⊗', 'darkspots(x)', '→', 'cheetah(x)'],
 ['carnivore(x)', '⊗', 'tawny(x)', '⊗', 'blackstripes(x)', '→', 'tiger(x)'],
 ['ungulate(x)',
  '⊗',
  'longlegs(x)',
  '⊗',
  'longneck(x)',
  '⊗',
  'tawny(x)',
  '⊗',
  'darkspots(x)',
  '→',
  'giraffe(x)'],
 ['ungulate(x)', '⊗', 'white(x)', '⊗', 'blackstripes(x)', '→', 'zebra(x)'],
 ['bird(x)',
  '⊗',
  'longlegs(x)',
  '⊗',
  'longneck(x)',
  '⊗',
  'black(x)',
  '→',
  'ostrich(x)'],
 ['bird(x)', '⊗', 'swim(x)', '⊗', 'blackwhite(x)', '→', 'penguin(x)'],
 ['bird(x)', '⊗', 'goodflier(x)', '→', 'albatross(x)'],

In [10]:
knowledge_base[0]

['hair(x)', '→', 'mammal(x)']

In [11]:
f = knowledge_base[0]

convert_formula(f)

['¬', 'hair(x)', '⊕', 'mammal(x)']

In [12]:
# ここは挙動がおかしい 本来なら先頭の 否定 は取れているべき

negation(convert_formula(f))

['¬', 'hair(x)', '⊗', '¬', 'mammal(x)']

In [7]:
predicates = []

for formula in knowledge_base:
    for item in formula:
        if item not in symbols_1 and item not in predicates:
            predicates.append(item)

len_j = len(predicates)


w_j = cp.Variable(shape=(len_j, 3))
xi_h = cp.Variable(shape=(len_h, 1), nonneg=True)

predicates_dict = {predicate: Predicate(w_j[j, :]).func for j, predicate in enumerate(predicates)}

display(predicates_dict['hair(x)'])
display(predicates_dict.keys())

# knowledge_base_trans = []

# for formula in knowledge_base:
#     formula_trans = []

#     for item in formula:
#         if item in predicates_dict.keys():
#             formula_trans.append(predicates_dict[item])
#         else:
#             formula_trans.append(item)
    
#     knowledge_base_trans.append(formula_trans)

# knowledge_base_trans   

<bound method Predicate.func of <utils.misc.Predicate object at 0x7f9b04fa24a0>>

dict_keys(['hair(x)', 'mammal(x)', 'milk(x)', 'feather(x)', 'bird(x)', 'layeggs(x)', 'meat(x)', 'carnivore(x)', 'pointedteeth(x)', 'claws(x)', 'forwardeyes(x)', 'hoofs(x)', 'ungulate(x)', 'cud(x)', 'tawny(x)', 'darkspots(x)', 'cheetah(x)', 'blackstripes(x)', 'tiger(x)', 'longlegs(x)', 'longneck(x)', 'giraffe(x)', 'white(x)', 'zebra(x)', 'black(x)', 'ostrich(x)', 'swim(x)', 'blackwhite(x)', 'penguin(x)', 'goodflier(x)', 'albatross(x)'])

In [6]:
len_j = 3 
len_h = 2

w_j = cp.Variable(shape=(len_j, 3))

w_1 = w_j[0, :]
w_2 = w_j[1, :]
w_3 = w_j[2, :]

xi_h = cp.Variable(shape=(len_h, 1), nonneg=True)
xi_1, xi_2 = xi_h[0, :], xi_h[1, :]
xi_1, xi_2


p_1 = Predicate(w_1).func
p_2 = Predicate(w_2).func
p_3 = Predicate(w_3).func

x_test = np.random.rand(2)

In [7]:
w_4 = cp.Variable(shape=(3))
p_4 = Predicate(w_4).func

In [95]:
def formula_transformer(formula_decomposed):
    neg_num = count_neg(formula_decomposed)

    while neg_num > 0:
        target_index = get_first_neg_index(formula_decomposed)

        # 演算に使用する値を取得
        x = formula_decomposed[target_index + 1]

        # 演算の実行
        operation = symbols_1_semanticized['¬']
        result = operation(x)

        # 演算結果で置き換え，演算子（¬）の削除
        formula_decomposed[target_index + 1] = result
        formula_decomposed.pop(target_index)

        neg_num -= 1

    while len(formula_decomposed) > 1:
        target_index = None

        # 分割された formula （リスト）から最初の演算子のインデックスを取得
        for index, token in enumerate(formula_decomposed):
            if token in symbols_1_semanticized.keys():
                target_index = index
                break

        # 対応する演算子の関数を取得
        symbol = formula_decomposed[target_index]
        operation = symbols_1_semanticized[symbol]

        # 演算に使用する値のペアを取得
        x = formula_decomposed[target_index - 1]
        y = formula_decomposed[target_index + 1]

        # 実際に演算を行う
        result = operation(x, y)

        # 演算結果で置き換え，演算済み要素の削除

        formula_decomposed[target_index] = result
        indices_to_remove = [target_index - 1, target_index + 1]
        filtered = []

        for i, item in enumerate(formula_decomposed):
            if i not in indices_to_remove:
                filtered.append(item)

        formula_decomposed = filtered

    return formula_decomposed[0]

In [101]:
f_decomp = [0.5, '→', '¬', 0.3, '→', '¬', 0.3, '→', '¬', 0.3]
f_processed = formula_transformer(f_decomp)
f_processed.value

array(1.)

In [13]:
# f = "mammal(x) ⊗ pointedteeth(x) ⊗ claws(x) ⊗ forwardeyes(x) → carnivore(x)".split()
# f = [p_1(x_test), '⊗', p_2(x_test), '⊗', p_3(x_test), '→', p_4(x_test)]
# f = [p_1(x_test), '⊗', p_2(x_test), '→', p_3(x_test)]
f = [.4, '⊗', .2, '→', .3]
f

[0.4, '⊗', 0.2, '→', 0.3]

In [14]:
f = [.4, '⊗', .2, '→', .3]
print(f'original formula:')
display(f)
print()

def process_neg(formula):
    neg_num = count_neg(formula)

    while neg_num > 0:
        target_index = get_first_neg_index(formula)

        # 演算に使用する値を取得
        x = formula[target_index + 1]

        # 演算の実行
        operation = symbols_1_semanticized['¬']
        result = operation(x)

        # 演算結果で置き換え，演算子（¬）の削除
        formula[target_index + 1] = result
        formula.pop(target_index)

        neg_num -= 1

    # return formula


def process_formula(formula):
    process_neg(formula)

    count_(formula)



process_formula(f)

original formula:


[0.4, '⊗', 0.2, '→', 0.3]

In [5]:
# formula = knowledge_base[4]
formula = ['¬', '¬', '¬', '¬', '¬', 'mammal(x)', '⊕', '¬','¬', '¬', '¬',  'meat(x)', '⊕', '¬', 'carnivore(x)']


convert_formula(formula)

['¬', 'mammal(x)', '⊕', 'meat(x)', '⊕', '¬', 'carnivore(x)']

In [20]:
logical_constraints = []

for formula in knowledge_base:
    print(convert_formula(formula))

['¬', 'hair(x)', '⊕', 'mammal(x)']
['¬', 'milk(x)', '⊕', 'mammal(x)']
['¬', 'feather(x)', '⊕', 'bird(x)']
['¬', 'layeggs(x)', '⊕', 'bird(x)']
['¬', 'mammal(x)', '⊕', '¬', 'meat(x)', '⊕', 'carnivore(x)']
['¬', 'mammal(x)', '⊕', '¬', 'pointedteeth(x)', '⊕', '¬', 'claws(x)', '⊕', '¬', 'forwardeyes(x)', '⊕', 'carnivore(x)']
['¬', 'mammal(x)', '⊕', '¬', 'hoofs(x)', '⊕', 'ungulate(x)']
['¬', 'mammal(x)', '⊕', '¬', 'cud(x)', '⊕', 'ungulate(x)']
['¬', 'carnivore(x)', '⊕', '¬', 'tawny(x)', '⊕', '¬', 'darkspots(x)', '⊕', 'cheetah(x)']
['¬', 'carnivore(x)', '⊕', '¬', 'tawny(x)', '⊕', '¬', 'blackstripes(x)', '⊕', 'tiger(x)']
['¬', 'ungulate(x)', '⊕', '¬', 'longlegs(x)', '⊕', '¬', 'longneck(x)', '⊕', '¬', 'tawny(x)', '⊕', '¬', 'darkspots(x)', '⊕', 'giraffe(x)']
['¬', 'ungulate(x)', '⊕', '¬', 'white(x)', '⊕', '¬', 'blackstripes(x)', '⊕', 'zebra(x)']
['¬', 'bird(x)', '⊕', '¬', 'longlegs(x)', '⊕', '¬', 'longneck(x)', '⊕', '¬', 'black(x)', '⊕', 'ostrich(x)']
['¬', 'bird(x)', '⊕', '¬', 'swim(x)', '⊕', '

In [48]:
formula = ['¬', 'ungulate(x)', '⊕', '¬', 'longlegs(x)', '⊕', '¬', 'longneck(x)', '⊕', '¬', 'tawny(x)', '⊕', '¬', 'darkspots(x)', '⊕', 'giraffe(x)']
# formula = ['¬', 'feather(x)', '⊕', 'bird(x)']

target_indices = []
for i, item in enumerate(formula):
    if item == '⊕':
        target_indices.append(i)

# print(target_indices)

f_decomp = []
f_decomp.append([1.])
f_decomp.append(formula[:target_indices[0]])
for i in range(len(target_indices) - 1):
    # print(target_indices[i])
    start_idx = target_indices[i] + 1
    end_idx = target_indices[i+1]
    f = formula[start_idx:end_idx]
    f_decomp.append(f)

if type(formula[-1]) == list:
    f_decomp.append(formula[-1])
else:
    f_decomp.append([formula[-1]])

f_decomp

[[1.0],
 ['¬', 'ungulate(x)'],
 ['¬', 'longlegs(x)'],
 ['¬', 'longneck(x)'],
 ['¬', 'tawny(x)'],
 ['¬', 'darkspots(x)'],
 ['giraffe(x)']]

In [49]:
# ¬ 否定
def negation(x):
    if type(x) != list:
        return 1 - x
    else:
        formula = []
        for item in x:
            if item == '∧':
                formula.append('∨')
            elif item == '∨':
                formula.append('∧')
            elif item == '⊕':
                formula.append('⊗')
            elif item == '⊗':
                formula.append('⊕')
            elif item == '¬':
                formula.append('¬')
            else:
                formula.append('¬')
                formula.append(item)

        return formula


In [51]:
for formula in f_decomp:
    print(formula)
    print()
    print(negation(formula))
    print(convert_formula(negation(formula)))
    print()
    print()

[1.0]

['¬', 1.0]
['¬', 1.0]


['¬', 'ungulate(x)']

['¬', '¬', 'ungulate(x)']
['ungulate(x)']


['¬', 'longlegs(x)']

['¬', '¬', 'longlegs(x)']
['longlegs(x)']


['¬', 'longneck(x)']

['¬', '¬', 'longneck(x)']
['longneck(x)']


['¬', 'tawny(x)']

['¬', '¬', 'tawny(x)']
['tawny(x)']


['¬', 'darkspots(x)']

['¬', '¬', 'darkspots(x)']
['darkspots(x)']


['giraffe(x)']

['¬', 'giraffe(x)']
['¬', 'giraffe(x)']




In [56]:
tmp_knowledge_base = []

for formula in knowledge_base:
    tmp_knowledge_base.append(convert_formula(formula))

tmp_knowledge_base

[['¬', 'hair(x)', '⊕', 'mammal(x)'],
 ['¬', 'milk(x)', '⊕', 'mammal(x)'],
 ['¬', 'feather(x)', '⊕', 'bird(x)'],
 ['¬', 'layeggs(x)', '⊕', 'bird(x)'],
 ['¬', 'mammal(x)', '⊕', '¬', 'meat(x)', '⊕', 'carnivore(x)'],
 ['¬',
  'mammal(x)',
  '⊕',
  '¬',
  'pointedteeth(x)',
  '⊕',
  '¬',
  'claws(x)',
  '⊕',
  '¬',
  'forwardeyes(x)',
  '⊕',
  'carnivore(x)'],
 ['¬', 'mammal(x)', '⊕', '¬', 'hoofs(x)', '⊕', 'ungulate(x)'],
 ['¬', 'mammal(x)', '⊕', '¬', 'cud(x)', '⊕', 'ungulate(x)'],
 ['¬',
  'carnivore(x)',
  '⊕',
  '¬',
  'tawny(x)',
  '⊕',
  '¬',
  'darkspots(x)',
  '⊕',
  'cheetah(x)'],
 ['¬',
  'carnivore(x)',
  '⊕',
  '¬',
  'tawny(x)',
  '⊕',
  '¬',
  'blackstripes(x)',
  '⊕',
  'tiger(x)'],
 ['¬',
  'ungulate(x)',
  '⊕',
  '¬',
  'longlegs(x)',
  '⊕',
  '¬',
  'longneck(x)',
  '⊕',
  '¬',
  'tawny(x)',
  '⊕',
  '¬',
  'darkspots(x)',
  '⊕',
  'giraffe(x)'],
 ['¬',
  'ungulate(x)',
  '⊕',
  '¬',
  'white(x)',
  '⊕',
  '¬',
  'blackstripes(x)',
  '⊕',
  'zebra(x)'],
 ['¬',
  'bird(x)',


In [65]:
# converted_knowledge_base = []
converted_knowledge_base = []

for formula in tmp_knowledge_base:

    target_indices = []
    for i, item in enumerate(formula):
        if item == '⊕':
            target_indices.append(i)

    # print(target_indices)

    f_decomp = []
    f_decomp.append(formula[:target_indices[0]])
    for i in range(len(target_indices) - 1):
        # print(target_indices[i])
        start_idx = target_indices[i] + 1
        end_idx = target_indices[i+1]
        f = formula[start_idx:end_idx]
        f_decomp.append(f)

    if type(formula[-1]) == list:
        f_decomp.append(formula[-1])
    else:
        f_decomp.append([formula[-1]])

    converted_knowledge_base.append(f_decomp)

print(len(converted_knowledge_base))

    # for f in f_decomp:
        # converted_knowledge_base.append(convert_formula(negation(f)))
        

32


In [66]:
for formula in converted_knowledge_base:
    print(formula)

[['¬', 'hair(x)'], ['mammal(x)']]
[['¬', 'milk(x)'], ['mammal(x)']]
[['¬', 'feather(x)'], ['bird(x)']]
[['¬', 'layeggs(x)'], ['bird(x)']]
[['¬', 'mammal(x)'], ['¬', 'meat(x)'], ['carnivore(x)']]
[['¬', 'mammal(x)'], ['¬', 'pointedteeth(x)'], ['¬', 'claws(x)'], ['¬', 'forwardeyes(x)'], ['carnivore(x)']]
[['¬', 'mammal(x)'], ['¬', 'hoofs(x)'], ['ungulate(x)']]
[['¬', 'mammal(x)'], ['¬', 'cud(x)'], ['ungulate(x)']]
[['¬', 'carnivore(x)'], ['¬', 'tawny(x)'], ['¬', 'darkspots(x)'], ['cheetah(x)']]
[['¬', 'carnivore(x)'], ['¬', 'tawny(x)'], ['¬', 'blackstripes(x)'], ['tiger(x)']]
[['¬', 'ungulate(x)'], ['¬', 'longlegs(x)'], ['¬', 'longneck(x)'], ['¬', 'tawny(x)'], ['¬', 'darkspots(x)'], ['giraffe(x)']]
[['¬', 'ungulate(x)'], ['¬', 'white(x)'], ['¬', 'blackstripes(x)'], ['zebra(x)']]
[['¬', 'bird(x)'], ['¬', 'longlegs(x)'], ['¬', 'longneck(x)'], ['¬', 'black(x)'], ['ostrich(x)']]
[['¬', 'bird(x)'], ['¬', 'swim(x)'], ['¬', 'blackwhite(x)'], ['penguin(x)']]
[['¬', 'bird(x)'], ['¬', 'goodflier(x

In [72]:
predicates = []

for formula in knowledge_base:
    for item in formula:
        if item not in symbols_1 and item not in predicates:
            predicates.append(item)

len_j = len(predicates)
xi_h = cp.Variable(shape=(len_h * 2, 1), nonneg=True)

w_j = cp.Variable(shape=(len_j, 3))
predicates_dict = {predicate: Predicate(w_j[j, :]).func for j, predicate in enumerate(predicates)}

In [ ]:
logical_constraints = []

for i, formula in enumerate(converted_knowledge_base):
    for formula_tiny in formula:
        
    logical_constraints += [
        negation(1.) <= xi_h[2*i],
        convert_formula() 

    ]

In [71]:
tmp = [1, 2]

while 

0

In [79]:
for i in converted_knowledge_base:
    for j in i:
        if len(j) > 3:
            print('yey')

In [70]:
converted_knowledge_base

[[['¬', 'hair(x)'], ['mammal(x)']],
 [['¬', 'milk(x)'], ['mammal(x)']],
 [['¬', 'feather(x)'], ['bird(x)']],
 [['¬', 'layeggs(x)'], ['bird(x)']],
 [['¬', 'mammal(x)'], ['¬', 'meat(x)'], ['carnivore(x)']],
 [['¬', 'mammal(x)'],
  ['¬', 'pointedteeth(x)'],
  ['¬', 'claws(x)'],
  ['¬', 'forwardeyes(x)'],
  ['carnivore(x)']],
 [['¬', 'mammal(x)'], ['¬', 'hoofs(x)'], ['ungulate(x)']],
 [['¬', 'mammal(x)'], ['¬', 'cud(x)'], ['ungulate(x)']],
 [['¬', 'carnivore(x)'],
  ['¬', 'tawny(x)'],
  ['¬', 'darkspots(x)'],
  ['cheetah(x)']],
 [['¬', 'carnivore(x)'],
  ['¬', 'tawny(x)'],
  ['¬', 'blackstripes(x)'],
  ['tiger(x)']],
 [['¬', 'ungulate(x)'],
  ['¬', 'longlegs(x)'],
  ['¬', 'longneck(x)'],
  ['¬', 'tawny(x)'],
  ['¬', 'darkspots(x)'],
  ['giraffe(x)']],
 [['¬', 'ungulate(x)'],
  ['¬', 'white(x)'],
  ['¬', 'blackstripes(x)'],
  ['zebra(x)']],
 [['¬', 'bird(x)'],
  ['¬', 'longlegs(x)'],
  ['¬', 'longneck(x)'],
  ['¬', 'black(x)'],
  ['ostrich(x)']],
 [['¬', 'bird(x)'], ['¬', 'swim(x)'], ['¬', 

In [67]:
predicates = []

for formula in knowledge_base:
    for item in formula:
        if item not in symbols_1 and item not in predicates:
            predicates.append(item)

len_j = len(predicates)


w_j = cp.Variable(shape=(len_j, 3))
# xi_h = cp.Variable(shape=(len_h, 1), nonneg=True)

predicates_dict = {predicate: Predicate(w_j[j, :]).func for j, predicate in enumerate(predicates)}

display(predicates_dict['hair(x)'])
display(predicates_dict.keys())

knowledge_base_trans = []

for formula in knowledge_base:
    formula_trans = []

    for item in formula:
        if item in predicates_dict.keys():
            formula_trans.append(predicates_dict[item])
        else:
            formula_trans.append(item)
    
    knowledge_base_trans.append(formula_trans)

knowledge_base_trans   

<bound method Predicate.func of <utils.misc.Predicate object at 0x7fc266684310>>

dict_keys(['hair(x)', 'mammal(x)', 'milk(x)', 'feather(x)', 'bird(x)', 'layeggs(x)', 'meat(x)', 'carnivore(x)', 'pointedteeth(x)', 'claws(x)', 'forwardeyes(x)', 'hoofs(x)', 'ungulate(x)', 'cud(x)', 'tawny(x)', 'darkspots(x)', 'cheetah(x)', 'blackstripes(x)', 'tiger(x)', 'longlegs(x)', 'longneck(x)', 'giraffe(x)', 'white(x)', 'zebra(x)', 'black(x)', 'ostrich(x)', 'swim(x)', 'blackwhite(x)', 'penguin(x)', 'goodflier(x)', 'albatross(x)'])

[[<bound method Predicate.func of <utils.misc.Predicate object at 0x7fc266684310>>,
  '→',
  <bound method Predicate.func of <utils.misc.Predicate object at 0x7fc2666863e0>>],
 [<bound method Predicate.func of <utils.misc.Predicate object at 0x7fc266684f40>>,
  '→',
  <bound method Predicate.func of <utils.misc.Predicate object at 0x7fc2666863e0>>],
 [<bound method Predicate.func of <utils.misc.Predicate object at 0x7fc2666870a0>>,
  '→',
  <bound method Predicate.func of <utils.misc.Predicate object at 0x7fc266bd3a30>>],
 [<bound method Predicate.func of <utils.misc.Predicate object at 0x7fc266bd3610>>,
  '→',
  <bound method Predicate.func of <utils.misc.Predicate object at 0x7fc266bd3a30>>],
 [<bound method Predicate.func of <utils.misc.Predicate object at 0x7fc2666863e0>>,
  '⊗',
  <bound method Predicate.func of <utils.misc.Predicate object at 0x7fc266441de0>>,
  '→',
  <bound method Predicate.func of <utils.misc.Predicate object at 0x7fc266697940>>],
 [<bound method Predicate.fun

In [15]:
logical_constraints = []





array([[0.1, 0.5],
       [0.3, 0.7],
       [0.5, 0.4],
       [0.8, 0.3],
       [0.9, 0.2],
       [1. , 0.5]])